#RANDOM FOREST

#Librerias

In [1]:
# !pip install openpyxl
import pandas as pd
import numpy as np
from sklearn.preprocessing import StandardScaler
from sklearn.ensemble import IsolationForest
from statsmodels.tsa.seasonal import STL
import plotly.graph_objects as go
from sklearn.cluster import DBSCAN
from sklearn.svm import OneClassSVM

##Funciones

##Llenar todos los datos

In [2]:
url = 'https://github.com/anfisbena/MIAD/raw/main/GPA/FINAL%20CONTUGAS/Datos.xlsx'
df_inicial = pd.concat([pd.read_excel(url, sheet_name=name).assign(Cliente=name)
                    for name in pd.ExcelFile(url).sheet_names], ignore_index=True)

Cliente = ['CLIENTE1','CLIENTE2','CLIENTE3','CLIENTE4','CLIENTE5','CLIENTE6','CLIENTE7',
            'CLIENTE8','CLIENTE9','CLIENTE10','CLIENTE11','CLIENTE12','CLIENTE13','CLIENTE14',
            'CLIENTE15','CLIENTE16','CLIENTE17','CLIENTE18','CLIENTE19','CLIENTE20']


df_inicial['Fecha'] = pd.to_datetime(df_inicial['Fecha'])

##Funcion Tratar duplicados

In [3]:
def tratar_duplicados(df):
    """
    Trata registros duplicados por cliente y fecha aplicando estrategias específicas
    (promedio o mediana) por cliente y variable.
    """
    # Lista de clientes con su estrategia recomendada
    estrategias = {
        "CLIENTE2": "mediana",
        "CLIENTE3": "promedio",
        "CLIENTE8": "mediana",
        "CLIENTE11": "promedio",
        "CLIENTE16": "mediana",
        "CLIENTE18": {
            "Presion": "promedio", 
            "Temperatura": "promedio", 
            "Volumen": "mediana"
        }
    }

    # Filtrar solo los clientes con duplicados conocidos
    clientes_duplicados = estrategias.keys()
    df_filtrado = df[df['Cliente'].isin(clientes_duplicados)]

    # Agrupar y aplicar estrategia
    grupos = df_filtrado.groupby(['Cliente', 'Fecha'])

    filas_limpias = []

    for (cliente, fecha), grupo in grupos:
        if len(grupo) == 1:
            filas_limpias.append(grupo.iloc[0])
        else:
            estrategia = estrategias[cliente]
            if isinstance(estrategia, dict):
                fila = {
                    "Fecha": fecha,
                    "Cliente": cliente,
                    "Presion": grupo["Presion"].mean() if estrategia["Presion"] == "promedio" else grupo["Presion"].median(),
                    "Temperatura": grupo["Temperatura"].mean() if estrategia["Temperatura"] == "promedio" else grupo["Temperatura"].median(),
                    "Volumen": grupo["Volumen"].mean() if estrategia["Volumen"] == "promedio" else grupo["Volumen"].median()
                }
            else:
                func = grupo.median if estrategia == "mediana" else grupo.mean
                fila = func(numeric_only=True)
                fila["Fecha"] = fecha
                fila["Cliente"] = cliente
            filas_limpias.append(pd.Series(fila))

    # Crear dataFrame limpio solo con registros duplicados tratados
    df_sin_duplicados = pd.DataFrame(filas_limpias)

    # Eliminar duplicados del original y unir con la versión limpia
    df_final = pd.concat([
        df[~df.set_index(["Cliente", "Fecha"]).index.isin(df_sin_duplicados.set_index(["Cliente", "Fecha"]).index)],
        df_sin_duplicados
    ], ignore_index=True)

    return df_final

##Funcion crear y llenar campos horarios inexistentes


In [4]:
def tratar_inexistentes(df):
    """
    Rellena registros faltantes por cliente a un índice horario completo y aplica
    interpolación lineal a las variables numéricas.
    """
    # Diccionario para guardar los nuevos DataFrames reindexados por cliente
    clientes_reindexados = {}

    # Obtener la lista única de clientes del df_final
    clientes = df['Cliente'].unique()

    # Procesar cliente por cliente
    for cliente in clientes:
        df_cliente = df[df['Cliente'] == cliente].copy()

        # Poner 'Fecha' en formato correcto y como índice
        df_cliente = df_cliente.set_index('Fecha')

        # Obtener el rango de fechas del cliente
        fecha_min = df_cliente.index.min()
        fecha_max = df_cliente.index.max()

        # Crear indice horario
        nuevo_index = pd.date_range(start=fecha_min, end=fecha_max, freq="h")

        # Reindexar al nuevo índice
        df_cliente = df_cliente.reindex(nuevo_index)

        # Restaurar columna de cliente (porque puede perderse al reindexar)
        df_cliente['Cliente'] = cliente

        # Guardar en diccionario
        clientes_reindexados[cliente] = df_cliente

    # Concatenar todos los clientes
    df_completo = pd.concat(clientes_reindexados.values())
    df_completo.reset_index(inplace=True)
    df_completo.rename(columns={'index': 'Fecha'}, inplace=True)

    # Interpolación por cliente para las variables numéricas
    variables = ['Volumen', 'Presion', 'Temperatura']
    for var in variables:
        df_completo[var] = df_completo.groupby('Cliente')[var].transform(lambda x: x.interpolate())

    return df_completo

###Eliminar tendencia y dejar solo la estacionalidad del temperatura(Karine):

In [5]:
def eliminar_tendencia(df, columna):
    """
    Aplica la descomposición STL para eliminar la tendencia de una serie temporal por cliente.
    """
    df_stl = pd.DataFrame()

    for cliente in df['Cliente'].unique():
        df_cliente = df[df['Cliente'] == cliente].copy()

        if df_cliente[columna].isna().sum() > 0 or len(df_cliente) < 48:
            continue  # Evita clientes con demasiados NaN o pocos datos

        try:
            # Descomposición STL: eliminamos la tendencia (trend) y mantenemos la estacionalidad
            stl = STL(df_cliente[columna], period=24)  # Periodo diario para datos horarios
            resultado = stl.fit()

            # Eliminar la tendencia (restar la tendencia)
            df_cliente[f'{columna}SinTendencia'] = df_cliente[columna] - resultado.trend
        except Exception as e:
            print(f"Error procesando cliente {cliente}: {e}")
            continue

        df_cliente = df_cliente.reset_index()
        df_stl = pd.concat([df_stl, df_cliente], ignore_index=True)

    return df_stl

### Tratar Datos, interpolarlos y completar todo el DF (Diego)

In [6]:
def escalar_datos(df):
    """
    Escala las variables numéricas usando escaladores robustos.
    """
    df_resultado = df.copy()

    for cliente_id, cliente_data in df.groupby('Cliente'):
        # Extraer variables numéricas
        features = cliente_data[['Presion', 'TemperaturaSinTendencia', 'Volumen']].astype('float32')

        # Escalar
        scaler = StandardScaler()
        features_scaled = scaler.fit_transform(features)

        # Asignar columnas escaladas
        df_resultado.loc[cliente_data.index, 'Presion_scaled'] = features_scaled[:, 0]
        df_resultado.loc[cliente_data.index, 'Temperatura_scaled'] = features_scaled[:, 1]
        df_resultado.loc[cliente_data.index, 'Volumen_scaled'] = features_scaled[:, 2]

    return df_resultado

In [7]:
def crear_findings_norma(df):
    df=df.copy()
    # ---- 1. Detección por Reglas de Negocio ---- #
    df['alerta_presion'] = ~df['Presion'].between(7.25, 17.4)
    df['alerta_temperatura'] = ~df['Temperatura'].between(0, 50)
    # Crear una condición para detectar cuando alguna variable es 0 y las otras > 0
    condicion_cero =(
        ((df['Presion'] == 0) & (df['Temperatura'] > 0) & (df['Volumen'] > 0)) |  # presión es 0
        ((df['Temperatura'] == 0) & (df['Presion'] > 0) & (df['Volumen'] > 0))   # temperatura es 0
    )  # volumen es 0
    # # Combinar todas las alertas
    df['alerta_reglas'] = df['alerta_presion'] | df['alerta_temperatura'] | condicion_cero
    return df

In [8]:
df=tratar_duplicados(df_inicial)
df.head()

,Fecha,Presion,Temperatura,Volumen,Cliente
0,2019-01-14 00:00:00,17.732563,28.209354,20.969751,CLIENTE1
1,2019-01-14 01:00:00,17.747776,28.518614,17.845739,CLIENTE1
2,2019-01-14 02:00:00,17.758916,28.230191,20.975914,CLIENTE1
3,2019-01-14 03:00:00,17.727940,27.811509,20.592299,CLIENTE1
4,2019-01-14 04:00:00,17.746484,27.795293,21.690626,CLIENTE1


In [9]:
df=tratar_inexistentes(df)
df.head()


,Fecha,Presion,Temperatura,Volumen,Cliente
0,2019-01-14 00:00:00,17.732563,28.209354,20.969751,CLIENTE1
1,2019-01-14 01:00:00,17.747776,28.518614,17.845739,CLIENTE1
2,2019-01-14 02:00:00,17.758916,28.230191,20.975914,CLIENTE1
3,2019-01-14 03:00:00,17.727940,27.811509,20.592299,CLIENTE1
4,2019-01-14 04:00:00,17.746484,27.795293,21.690626,CLIENTE1


In [10]:
# df=crear_findings_norma(df)
df=eliminar_tendencia(df,'Temperatura')
df.head()

,index,Fecha,Presion,Temperatura,Volumen,Cliente,TemperaturaSinTendencia
0,0,2019-01-14 00:00:00,17.732563,28.209354,20.969751,CLIENTE1,0.575216
1,1,2019-01-14 01:00:00,17.747776,28.518614,17.845739,CLIENTE1,0.893297
2,2,2019-01-14 02:00:00,17.758916,28.230191,20.975914,CLIENTE1,0.613994
3,3,2019-01-14 03:00:00,17.727940,27.811509,20.592299,CLIENTE1,0.204748
4,4,2019-01-14 04:00:00,17.746484,27.795293,21.690626,CLIENTE1,0.198320


In [11]:
df=escalar_datos(df)
df=df.rename(columns={"Fecha": "timestamp",'Presion':'presion','Volumen':'volumen','Temperatura':'temperatura','Cliente':'cliente_id'})
df.head()

,index,timestamp,presion,temperatura,volumen,cliente_id,TemperaturaSinTendencia,Presion_scaled,Temperatura_scaled,Volumen_scaled
0,0,2019-01-14 00:00:00,17.732563,28.209354,20.969751,CLIENTE1,0.575216,0.548186,0.759369,0.123697
1,1,2019-01-14 01:00:00,17.747776,28.518614,17.845739,CLIENTE1,0.893297,0.590668,1.179327,-0.269701
2,2,2019-01-14 02:00:00,17.758916,28.230191,20.975914,CLIENTE1,0.613994,0.621778,0.810566,0.124473
3,3,2019-01-14 03:00:00,17.727940,27.811509,20.592299,CLIENTE1,0.204748,0.535276,0.270247,0.076165
4,4,2019-01-14 04:00:00,17.746484,27.795293,21.690626,CLIENTE1,0.198320,0.587057,0.261760,0.214475


###Modelo ISOFOREST

#EJECUCION

In [14]:
from sklearn.metrics import confusion_matrix

Cliente ={
    "CLIENTE1":{'eps':0.5,'min_samp':5,'mf':0.8,'nt':200,'cnt':0.01},
    "CLIENTE2":{'eps':0.5,'min_samp':5,'mf':0.8,'nt':50,'cnt':0.01},
    "CLIENTE3":{'eps':0.5,'min_samp':5,'mf':0.8,'nt':100,'cnt':0.01},
    "CLIENTE4":{'eps':0.5,'min_samp':5,'mf':0.8,'nt':200,'cnt':0.01},
    "CLIENTE5":{'eps':0.5,'min_samp':5,'mf':0.8,'nt':500,'cnt':0.01},
    "CLIENTE6":{'eps':0.5,'min_samp':5,'mf':0.8,'nt':100,'cnt':0.01},
    "CLIENTE7":{'eps':0.5,'min_samp':5,'mf':0.8,'nt':100,'cnt':0.01},
    "CLIENTE8":{'eps':0.5,'min_samp':5,'mf':0.5,'nt':100,'cnt':0.1},
    "CLIENTE9":{'eps':0.5,'min_samp':5,'mf':0.8,'nt':500,'cnt':0.05},
    "CLIENTE10":{'eps':0.5,'min_samp':5,'mf':0.8,'nt':200,'cnt':0.01},
    "CLIENTE11":{'eps':0.5,'min_samp':5,'mf':0.5,'nt':500,'cnt':0.01},
    "CLIENTE12":{'eps':0.5,'min_samp':5,'mf':0.8,'nt':50,'cnt':0.01},
    "CLIENTE13":{'eps':0.5,'min_samp':5,'mf':0.8,'nt':100,'cnt':0.05},
    "CLIENTE14":{'eps':0.5,'min_samp':5,'mf':0.5,'nt':500,'cnt':0.01},
    "CLIENTE15":{'eps':0.5,'min_samp':5,'mf':0.8,'nt':100,'cnt':0.05},
    "CLIENTE16":{'eps':0.5,'min_samp':5,'mf':0.5,'nt':500,'cnt':0.1},
    "CLIENTE17":{'eps':0.5,'min_samp':5,'mf':0.8,'nt':200,'cnt':0.01},
    "CLIENTE18":{'eps':0.5,'min_samp':5,'mf':0.8,'nt':100,'cnt':0.01},
    "CLIENTE19":{'eps':0.5,'min_samp':5,'mf':0.8,'nt':50,'cnt':0.1},
    "CLIENTE20":{'eps':0.5,'min_samp':5,'mf':0.8,'nt':200,'cnt':0.01},
}

def detectar_anomalias_IQR(serie):
    Q1 = serie.quantile(0.25)
    Q3 = serie.quantile(0.75)
    IQR = Q3 - Q1
    lim_inf = Q1 - 1.5 * IQR
    lim_sup = Q3 + 1.5 * IQR
    return ~serie.between(lim_inf, lim_sup)

def entrenar_por_cliente(df):
    # Realizar conexión a la BD

    df_all = df.copy()
    df_all['timestamp'] = pd.to_datetime(df_all['timestamp'])

    resultados = []
    clientes_excluidos = {19, 4, 20, 6, 1, 17, 5, 14, 18, 2}

    for cliente in Cliente.keys():
        df = df_all[df_all['cliente_id'] == cliente].copy()
        df.set_index('timestamp', inplace=True)

        # Inicializar las columnas de alerta en False por defecto
        df['alerta_presion'] = False
        df['alerta_temperatura'] = False

        # ---- 1. Detección por Reglas de Negocio (IQR) en datos originales ---- #
        if cliente not in clientes_excluidos:
            df['alerta_presion'] = detectar_anomalias_IQR(df['presion'])
            df['alerta_temperatura'] = detectar_anomalias_IQR(df['temperatura'])


        # Combinar todas las alertas
        df['alerta_reglas'] = df['alerta_presion'] | df['alerta_temperatura']
        #| condicion_cero

        # ---- 2. Detección por Modelos de ML ---- #
        features = ['Presion_scaled', 'Temperatura_scaled', 'Volumen_scaled']
        model = DBSCAN(min_samples=Cliente[cliente]['min_samp'], eps=Cliente[cliente]['eps'])
        dbscan_pred = model.fit_predict(df[features])
        iso = IsolationForest( n_estimators=Cliente[cliente]['nt'], contamination=Cliente[cliente]['cnt'], max_features=Cliente[cliente]['mf'], random_state=42)
        iso_pred = iso.fit_predict(df[features])
        one_class_svm = OneClassSVM(nu=0.01, kernel="rbf", gamma="scale")
        svm_pred = one_class_svm.fit_predict(df[features])
        df['Anomalia_iso'] = np.where(iso_pred == -1, True, False)
        df['Anomalia_DBSCAN'] = np.where(dbscan_pred == -1, True, False)
        df['Anomalia_SVM'] = np.where(svm_pred == -1, True, False)
        df['Anomalia_general']= df['Anomalia_DBSCAN'] | df['Anomalia_SVM'] | df['Anomalia_iso']

        # ---- 3. Severidad Combinada (Reglas + ML) ---- #
        df['severidad'] = df.apply(lambda row: 
            'Alto' if row['alerta_reglas'] == 1
            else 'Potencial' if row['Anomalia_general'] == 1  
            else 'OK', axis=1)


        # Guardar resultados
        resultados.append(df)

    # Unir resultados
    df_resultado = pd.concat(resultados)
    df_resultado.drop(columns=['index'], errors='ignore', inplace=True)
    df_resultado.reset_index(inplace=True)

    return df_resultado


def matriz_confusion_por_modelo(df_resultado):
    y_true = df_resultado['alerta_reglas'].astype(int)

    modelos = ['Anomalia_iso', 'Anomalia_DBSCAN', 'Anomalia_SVM','Anomalia_general']
    matrices = {}

    for modelo in modelos:
        y_pred = df_resultado[modelo].astype(int)
        matriz = confusion_matrix(y_true, y_pred, labels=[0, 1])
        matriz_df = pd.DataFrame(
            matriz,
            index=['Real 0', 'Real 1'],
            columns=['Pred 0', 'Pred 1']
        )
        print(f"\n📊 Matriz de Confusión: {modelo}")
        print(matriz_df)
        matrices[modelo] = matriz_df

    return matrices

## correr modelos

In [15]:
df_resultado = entrenar_por_cliente(df)
matriz_confusion_por_modelo(df_resultado)

c:\Users\ANDRES\miniconda3\envs\opt\lib\site-packages\sklearn\base.py:439: UserWarning: X does not have valid feature names, but IsolationForest was fitted with feature names
  warnings.warn(
c:\Users\ANDRES\miniconda3\envs\opt\lib\site-packages\sklearn\base.py:439: UserWarning: X does not have valid feature names, but IsolationForest was fitted with feature names
  warnings.warn(
c:\Users\ANDRES\miniconda3\envs\opt\lib\site-packages\sklearn\base.py:439: UserWarning: X does not have valid feature names, but IsolationForest was fitted with feature names
  warnings.warn(
c:\Users\ANDRES\miniconda3\envs\opt\lib\site-packages\sklearn\base.py:439: UserWarning: X does not have valid feature names, but IsolationForest was fitted with feature names
  warnings.warn(
c:\Users\ANDRES\miniconda3\envs\opt\lib\site-packages\sklearn\base.py:439: UserWarning: X does not have valid feature names, but IsolationForest was fitted with feature names
  warnings.warn(
c:\Users\ANDRES\miniconda3\envs\opt\lib\


📊 Matriz de Confusión: Anomalia_iso
        Pred 0  Pred 1
Real 0  780848   14499
Real 1   63704   11189

📊 Matriz de Confusión: Anomalia_DBSCAN
        Pred 0  Pred 1
Real 0  794235    1112
Real 1   73395    1498

📊 Matriz de Confusión: Anomalia_SVM
        Pred 0  Pred 1
Real 0  790552    4795
Real 1   70683    4210

📊 Matriz de Confusión: Anomalia_general
        Pred 0  Pred 1
Real 0  777536   17811
Real 1   61913   12980


{'Anomalia_iso':         Pred 0  Pred 1
 Real 0  780848   14499
 Real 1   63704   11189,
 'Anomalia_DBSCAN':         Pred 0  Pred 1
 Real 0  794235    1112
 Real 1   73395    1498,
 'Anomalia_SVM':         Pred 0  Pred 1
 Real 0  790552    4795
 Real 1   70683    4210,
 'Anomalia_general':         Pred 0  Pred 1
 Real 0  777536   17811
 Real 1   61913   12980}